<a href="https://colab.research.google.com/github/eeroolli/SalsaAnnotation/blob/development/video_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import cv2, os
import csv

# libraries needed from outside
import os
from pathlib import Path
from os import mkdir
from os.path import exists, join, basename, splitext
import sys
import glob                       # help for finding files
from re import split              # regular expression string splitter
import subprocess                 #subprocess wraps around regular os commands
#import shutil                     # used for copying files in the os.
import pandas as pd
!pip install validators
import validators                 #for URL validation
#from validators import ValidationFailure


from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [30]:
root_path = "/content/gdrive/MyDrive/DanceApp/"



video_path = os.path.join(root_path,"1P_input_original")
data_output_path = os.path.join(root_path,"1P_input_original")
video_csv = os.path.join(data_output_path, "video_info.csv")
script_path = "/content/gdrive/MyDrive/2021-DSR-Porfolio/python_scripts"
# one cannot add items to sys.path that have a space in them and script_path must be
# the same for Ana and Eero.  Eero has now added a link from his google DSR to Ana's google 2021-DSR-Porfolio/

print("Using ", root_path, " as root_path")
print("Using ", data_output_path, " as data_output_path")
print("Using ", video_csv, " as video_csv")

def check_path(x):
  if os.path.isdir(x):
    print("Using", x )
  else:
    print("Problem: There was no ", x)  # No error keeps running

check_path(root_path)

def check_url(url_string: str):
    if validators.url(url_string):
      print("Using: ", url_string)
    else:
      print("Problem: The url is not valid ", url_string) 
      return False

# to select clip_names
video_size = "_h920"  # use format _h920

if os.path.isfile(video_csv):
  video_info = pd.read_csv(video_csv)
  print("The video_csv does already exist")
else:
  video_info = {"clip_name": [],
                "width": [] , 
                "height": [], 
                "frames_per_s": [],
                "usage":" "  ,
                "comment":" ",
                 "video_name": [], 
                }

video_info_df = pd.DataFrame(video_info)

Using  /content/gdrive/MyDrive/DanceApp/  as root_path
Using  /content/gdrive/MyDrive/DanceApp/1P_input_original  as data_output_path
Using  /content/gdrive/MyDrive/DanceApp/1P_input_original/video_info.csv  as video_csv
Using /content/gdrive/MyDrive/DanceApp/
The video_csv does already exist


In [31]:
    print(os.getcwd())
    video_list = glob.glob(os.path.join(video_path, "1*.mp4"))
    video_list = video_list + glob.glob(os.path.join(video_path, "1*.MOV"))
    paths_h920_video = glob.glob(os.path.join(video_path, "h920", "1*.mp4"))
    video_list = sorted(video_list)
    print("We have ", len(video_list), "1p_original videos")
    print("and ", len(paths_h920_video), " resized videos")

/content
We have  31 1p_original videos
and  26  resized videos


In [32]:
video_list

['/content/gdrive/MyDrive/DanceApp/1P_input_original/1P-Ana.mp4',
 '/content/gdrive/MyDrive/DanceApp/1P_input_original/1P-Person1-whole.mp4',
 '/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_Ana_c1.mp4',
 '/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_Ana_c2_20211212.mp4',
 '/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_Artur_05122021_chreo1.mp4',
 '/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_Dirk_choreo1_04.12.2021.mp4',
 '/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_Earl_05122021_choreo1_flipped_lr.mp4',
 '/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_Eero_1_05122021_choreo1.mp4',
 '/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_Eero_4_05122021_choreo1.mp4',
 '/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_Eero_6_05122021_choreo1.mp4',
 '/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_Eero_7_05122021_choreo1.mp4',
 '/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_Elizabet.mp4',
 '/content/gdrive/MyDrive/Dance

In [33]:
def get_video_info(video_name):
    #video_name is the full adress to a video
    clip_name = split("/", splitext(video_name)[0])[-1]
    video_part_name, video_ext = splitext(clip_name)
    # print("clip_name: ", clip_name)
    # print("video_name: ", video_name, " extention: ", video_ext)

    if os.path.isfile(video_name):
      width_temp = !ffprobe -v 0 -of csv=p=0 -select_streams v:0 -hide_banner -show_entries stream=width $video_name
      height_temp = !ffprobe -v 0 -of csv=p=0 -select_streams v:0 -hide_banner -show_entries stream=height $video_name
      text = !ffprobe -v 0 -of csv=p=0 -select_streams v:0 -hide_banner -show_entries stream=r_frame_rate $video_name

      if len(text)>0:
        width = int(width_temp[0])
        height = int(height_temp[0])
        frames, seconds = text[0].split("/")
        fps = int(frames) / int(seconds)
        print(width, height, text)
    else:
      print("The video_in does not exist.")
      # raise Exception("No input video file")
    return clip_name, width, height, fps, video_name
  
# get_video_info('/content/gdrive/MyDrive/DanceApp/1P_input_original/1p_sarah_salsa_basics_choreo1.mp4')

In [34]:
for i in range(len(video_list)):
  video_name = video_list[i]
  clip_name, width, height, fps, video_name = get_video_info(video_name=video_name)
  info = {"clip_name": clip_name, "width": width, "height": height, "frames_per_s": fps,  "usage":""  , "comment":"", "video_name": video_name }
  video_info_df = video_info_df.append(info, ignore_index=True)

1920 1080 ['90000/2999', '']
640 352 ['30/1', '']
1280 720 ['30/1', '']
1280 720 ['90000/1', '']
1080 1920 ['30/1']
1920 1080 ['30000/1001']
720 1280 ['30/1']
1920 1080 ['88/3', '']
1920 1080 ['88/3', '']
1920 1080 ['88/3', '']
1920 1080 ['88/3', '']
1920 1080 ['30/1', '']
848 560 ['30/1']
848 560 ['30/1']
848 560 ['30/1']
848 560 ['30/1']
1920 1080 ['60/1', '']
1920 1080 ['30/1', '']
1080 1920 ['30/1']
1080 1920 ['30/1']
1280 720 ['30000/1001', '']
1280 720 ['30000/1001', '']
640 352 ['23/1', '']
1920 1080 ['88/3', '']
1920 1080 ['88/3', '']
1920 1080 ['88/3', '']
754 1318 ['60/1']
742 1320 ['60/1']
640 480 ['30000/1001']
640 480 ['30000/1001']
1080 1920 ['30/1']


In [35]:
video_info_df

,Unnamed: 0,Unnamed: 0.1,clip_name,width,height,frames_per_s,usage,comment,video_name
0,0.0,0.0,1P-Ana,1920.0,1080.0,30.010003,NaN,NaN,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
1,1.0,1.0,1P-Person1-whole,640.0,352.0,30.000000,NaN,NaN,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
2,2.0,2.0,1p_Ana_c1,1280.0,720.0,30.000000,NaN,NaN,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
3,3.0,3.0,1p_Ana_c2_20211212,1280.0,720.0,90000.000000,NaN,NaN,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
4,4.0,4.0,1p_Artur_05122021_chreo1,1080.0,1920.0,30.000000,NaN,NaN,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
...,...,...,...,...,...,...,...,...,...
88,NaN,NaN,1p_Vassil_1_11122021_Choreo1,754.0,1318.0,60.000000,,,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
89,NaN,NaN,1p_Vassil_2_11122021_Choreo1,742.0,1320.0,60.000000,,,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
90,NaN,NaN,1p_abtin_1_choreo1,640.0,480.0,29.970030,,,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
91,NaN,NaN,1p_abtin_2_choreo1,640.0,480.0,29.970030,,,/content/gdrive/MyDrive/DanceApp/1P_input_orig...


In [36]:
video_info_df.to_csv(video_csv, index=False)

In [37]:
?pd.to_csv()
  

Object `pd.to_csv` not found.


In [38]:
video_info_df

,Unnamed: 0,Unnamed: 0.1,clip_name,width,height,frames_per_s,usage,comment,video_name
0,0.0,0.0,1P-Ana,1920.0,1080.0,30.010003,NaN,NaN,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
1,1.0,1.0,1P-Person1-whole,640.0,352.0,30.000000,NaN,NaN,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
2,2.0,2.0,1p_Ana_c1,1280.0,720.0,30.000000,NaN,NaN,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
3,3.0,3.0,1p_Ana_c2_20211212,1280.0,720.0,90000.000000,NaN,NaN,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
4,4.0,4.0,1p_Artur_05122021_chreo1,1080.0,1920.0,30.000000,NaN,NaN,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
...,...,...,...,...,...,...,...,...,...
88,NaN,NaN,1p_Vassil_1_11122021_Choreo1,754.0,1318.0,60.000000,,,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
89,NaN,NaN,1p_Vassil_2_11122021_Choreo1,742.0,1320.0,60.000000,,,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
90,NaN,NaN,1p_abtin_1_choreo1,640.0,480.0,29.970030,,,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
91,NaN,NaN,1p_abtin_2_choreo1,640.0,480.0,29.970030,,,/content/gdrive/MyDrive/DanceApp/1P_input_orig...
